In [ ]:
!pip install librosa transformers language-tool-python tensorflow matplotlib

In [ ]:
import pandas as pd
import numpy as np
import librosa
from transformers import pipeline
import language_tool_python

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [ ]:
def extract_audio_features(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    features = {
        'mfcc': librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40),
        'chroma': librosa.feature.chroma_stft(y=y, sr=sr),
        'contrast': librosa.feature.spectral_contrast(y=y, sr=sr)
    }
    return {k: np.mean(v, axis=1) for k, v in features.items()}

In [ ]:
asr_pipe = pipeline("automatic-speech-recognition",
                   model="openai/whisper-medium",
                   device="cuda" if torch.cuda.is_available() else "cpu")
def transcribe_audio(file_path):
  return asr_pipe(file_path)["text"]


In [ ]:
tool = language_tool_python.LanguageTool('en-US')

def analyze_grammar(text):
    matches = tool.check(text)
    return {
        'error_count': len(matches),
        'error_types': {type(m.ruleId) for m in matches}
    }

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Combine audio features, text statistics, and grammar metrics
preprocessor = ColumnTransformer([
    ('audio', StandardScaler(), audio_features_columns),
    ('text', StandardScaler(), text_features_columns)
])

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout

# Audio branch
audio_input = Input(shape=(audio_feature_dim,))
audio_dense = Dense(128, activation='relu')(audio_input)

# Text branch
text_input = Input(shape=(text_feature_dim,))
text_dense = Dense(128, activation='relu')(text_input)

# Fusion
merged = Concatenate()([audio_dense, text_dense])
output = Dense(1, activation='linear')(merged)

model = Model(inputs=[audio_input, text_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

history = model.fit(
    [X_train_audio, X_train_text],
    y_train,
    validation_split=0.2,
    epochs=100,
    callbacks=[
        EarlyStopping(patience=10),
        ReduceLROnPlateau(factor=0.2, patience=5)
    ]
)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict([X_test_audio, X_test_text])
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.2f}")